In [4]:
import json
from os import listdir

In [94]:
def load_matchup_files(players_struct,player,matchup):
    round_files = players_struct[player][matchup]
    
    rounds = []
    for rnd in round_files:
        f = open(rnd)
        rounds.append(json.load(f))
        f.close()
    return rounds

def build_players_struct(base_dir):
    #first I need to figure out what all the folders are
    player_folders=listdir(base_dir)
    #from that, parse and figure out how many players there are
    indices = []
    for name in player_folders:
        pieces = name.split('player')
        indices.append(int(pieces[1]))
        indices.append(int(pieces[-1]))
    min_idx = min(indices)
    max_idx = max(indices)
    players = []
    for i in range(min_idx, max_idx+1):
        players.append('player'+str(i))

    #now with a list of players, if player is in player folder,
    #then it is to be included in the 
    players_struct = {};
    for player in players:
        players_struct[player] = {}
        for fldr in player_folders:
            if player in fldr:
                round_folders = listdir(base_dir+fldr)
                players_struct[player][fldr] = []
                for rnd in round_folders:
                    players_struct[player][fldr].append(base_dir+fldr+'/'+rnd)
    return players_struct
def get_matchup_scores(rounds, player_name, matchup_name):
    #can check if last character matches, if it does, reverse, otherwise, normal scoring
    a_score = 0
    b_score = 0
    a_strat = list(rounds[0]['player_a'].keys())[0]
    b_strat = list(rounds[0]['player_b'].keys())[0]

    for rnd in rounds:
        a_score += rnd['player_a'][a_strat]['player']['play']['my_score']
        b_score += rnd['player_b'][b_strat]['player']['play']['my_score']

    if player_name[-1]==matchup_name[-1]:
        return b_score, a_score
    else:
        return a_score, b_score

def get_all_player_scores(players_struct):
    players = list(players_struct.keys())
    all_match_scores = {}
    for player_name in players:
        all_match_scores = get_player_score(players_struct,player_name,all_match_scores)
    all_match_scores    
    return all_match_scores

def get_player_score(players_struct, player_name, all_match_scores):
    matchups = list(players_struct[player_name].keys())
    all_match_scores[player_name] = 0

    for matchup_name in matchups:
        rounds = load_matchup_files(players_struct,player_name, matchup_name)
        all_match_scores[player_name] += get_matchup_scores(rounds,player_name,matchup_name)[0]
    return all_match_scores

def get_matchup_strats(matchup,player_name,matchup_name):
    a_strat = list(matchup[0]['player_a'].keys())[0]
    b_strat = list(matchup[0]['player_b'].keys())[0]
    #need to check that player name is in the right order
    if player_name[-1]==matchup_name[-1]:
        return b_strat, a_strat
    else:
        return a_strat, b_strat

In [170]:
#base_dir = "/home/kennethmclarney/Documents/RustProjects/noisygames/test_runs/Year2022Month1Day21Hour17Min56Sec7/"
base_dir = "/home/kennethmclarney/Documents/RustProjects/noisygames/test_runs/Year2022Month1Day23Hour2Min26Sec33/"
#great, now I've got a nice setup to organize results.
players_struct = build_players_struct(base_dir)
all_scores = get_all_player_scores(players_struct)

In [195]:
#in the future this can be handled more carefully
strategies=['AlwaysDefect','GrimTrigger','TitForTat','RandomDefect']
strat_vs_strat = {}
for strat_i in strategies:
    strat_vs_strat[strat_i] = {}
    for strat_j in strategies:
        strat_vs_strat[strat_i][strat_j]=0

players = list(players_struct.keys())
for player_name in players:
    matchups = list(players_struct[player_name].keys())
    for matchup_name in matchups:
        matchup=load_matchup_files(players_struct, player_name, matchup_name)
        #determine score and get the strategies
        scores=get_matchup_scores(matchup, player_name, matchup_name)
        strats=get_matchup_strats(matchup, player_name, matchup_name)
        #huh, when it's the same, how do we want to handle it? I guess it's overall points scored.
        if strats[0] == strats[1]:
            strat_vs_strat[strats[0]][strats[1]]+=scores[0]
            strat_vs_strat[strats[1]][strats[0]]+=scores[1]
        else:
            strat_vs_strat[strats[0]][strats[1]]+=scores[0]
strat_vs_strat

In [196]:
#display results

In [212]:
import matplotlib.pyplot as plt

In [ ]:
points_mtx = []
for idx_i in range(len(strategies)):
    points_mtx.append([])
    for idx_j in range(len(strategies)):
        points_mtx[-1].append(strat_vs_strat[strategies[idx_i]][strategies[idx_j]])
    points_mtx[-1].append(sum(list(strat_vs_strat[strategies[idx_i]].values())))
points_mtx

In [235]:
row_strats=strategies
col_strats=row_strats.copy()
col_strats.append("Total")

In [314]:
fig, ax = plt.subplots()
ax.set_axis_off()
fig.set_size_inches(60,20)
tbl = plt.table( cellText = points_mtx,
                colLabels = col_strats,
                rowLabels = row_strats,
                rowColours = ['skyblue']*4,
                 colColours = ['skyblue']*5,
                 colWidths = [.04]*5,
                 cellLoc = 'center',
                 loc = 'upper left'
                )
tbl.auto_set_font_size(False)
tbl.set_fontsize(72)
tbl.scale(5,20)
ax.set_title('If Playing Row, Performance Against Column',
            fontweight = "bold", fontsize=84, loc='left' )

plt.savefig('RowVsCol_Payoffs.png',dpi = 100)